In [1]:
import os
import pandas as pd
import time

If LOCAL_SCRATCH_DIR environment variable is not set, this notebook accesses the ../data directory for temporary files.

In [2]:
DATA_DIR = os.getenv("LOCAL_SCRATCH_DIR", default="../data")
filename = os.path.join(DATA_DIR, "gene_info.tsv")
print("Filename:", filename)

Filename: ../data/gene_info.tsv


In [3]:
start = time.time()

In [4]:
column_names = ["GeneID", "Symbol", "Synonyms", "description", "type_of_gene", "#tax_id"]

genes = pd.read_csv(filename, usecols=column_names, dtype=str, sep="\t")
genes.rename(columns={"#tax_id": "tax_id"}, inplace=True)

In [5]:
genes.query("type_of_gene == 'protein-coding'", inplace=True)

In [6]:
groups = genes.groupby("tax_id").size().reset_index(name="count")

In [7]:
groups = groups.sort_values("count", ascending=False)

### Number of human protein-coding genes (tax_id = 9606)

In [8]:
groups.query("tax_id == '9606'")

,tax_id,count
35211,9606,20596


### Top 5 organisms with the most protein-coding genes

In [9]:
groups.head()

,tax_id,count
27193,4565,104033
24813,3708,96995
34557,90675,82686
35042,94328,68154
24530,3635,67632


In [10]:
end = time.time()

In [11]:
print(f"Pandas: {end - start:.1f} sec.")

Pandas: 105.9 sec.
